In [27]:
from ccxt import okex5,bitget,bybit,binanceusdm,Exchange,binance,bitbank,bitfinex2
import pandas as pd
from random import random,randint,uniform
from datetime import datetime,timedelta

In [2]:
def generate_open_order(exchange:Exchange):
    exchange.load_markets()
    data_list = [symbol for symbol in exchange.symbols if ':USDT' in symbol]
    random_open_order_symbols = [data_list[i] for i in sorted([int(random()*len(data_list)) for _ in range(25)])]
    entry_data = {}
    order_id_count = 1
    for symbol in random_open_order_symbols:
        side = randint(0,1)
        symbol_history_df = pd.DataFrame(exchange.fetch_ohlcv(symbol, '15m', limit=1000))
        symbol_min_value = exchange.markets[symbol]['limits']['amount']['min']
        symbol_max_value = exchange.markets[symbol]['limits']['amount']['max']
        if symbol_max_value == None or symbol_min_value == None:
            symbol_max_value, symbol_min_value = 100,500
        quantity = uniform(symbol_min_value,symbol_max_value)
        if quantity*symbol_history_df[4][1] > 1000:
            quantity = round(1000/symbol_history_df[4][1],4)
        if side == 0:
            most_lowest_price_index = symbol_history_df[4].idxmin()
            entry_data[symbol] = {
                'order_id': order_id_count,
                'side': 'buy',
                'entry_price': symbol_history_df[4][most_lowest_price_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_lowest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
            }
        else:
            most_highest_price_index = symbol_history_df[4].idxmax()
            entry_data[symbol] = {
                'order_id': order_id_count,
                'side': 'sell',
                'entry_price': symbol_history_df[4][most_highest_price_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_highest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
            }
        order_id_count += 1
    return entry_data

def Caculate_Unrealized_Pnl(open_orders_data:dict,exchange:Exchange):
    for symbol in open_orders_data:
        symbol_history_df = pd.DataFrame(exchange.fetch_ohlcv(symbol, '1m', limit=10))
        most_recent_price_index = symbol_history_df[4].idxmax()
        most_recent_price = symbol_history_df[4][most_recent_price_index]
        if open_orders_data[symbol]['side'] == 'buy':
            open_orders_data[symbol]['Unrealized'] = round((most_recent_price - open_orders_data[symbol]['entry_price'])*open_orders_data[symbol]['entry_quantity'],2)
        else:
            open_orders_data[symbol]['Unrealized'] = round((open_orders_data[symbol]['entry_price'] - most_recent_price)*open_orders_data[symbol]['entry_quantity'],2)
    return open_orders_data

def generate_close_order(exchange:Exchange,total_trade_num:int,win_trade_num:int,time_start:datetime,pick_up_number:int=10):
    time_end = datetime.now() - timedelta(days=1)
    time_end = time_end.isoformat()
    time_end = exchange.parse8601(time_end)
    time_start = exchange.parse8601(time_start.isoformat())
    exchange.load_markets()
    data_list = [symbol for symbol in exchange.symbols if ':USDT' in symbol]
    random_close_order_symbols = [data_list[i] for i in sorted([int(random()*len(data_list)) for _ in range(pick_up_number)])]
    order_id_count = 1
    close_order_data = {}
    while len(close_order_data) < total_trade_num:
        symbol = random_close_order_symbols[int(random()*len(random_close_order_symbols))]
        side = randint(0,1)
        symbol_history_df = []
        time_temp = time_start
        while time_temp < time_end:
            symbol_history_df.extend(exchange.fetch_ohlcv(symbol, '1h', time_temp, limit=1000))
            time_temp = symbol_history_df[-1][0] + 3600000
        symbol_history_df = pd.DataFrame(symbol_history_df)
        symbol_min_value = exchange.markets[symbol]['limits']['amount']['min']
        symbol_max_value = exchange.markets[symbol]['limits']['amount']['max']
        if symbol_max_value == None or symbol_min_value == None:
            symbol_max_value, symbol_min_value = 100,500
        quantity = uniform(symbol_min_value,symbol_max_value)
        if quantity*symbol_history_df[4][1] > 1000:
            quantity = round(1000/symbol_history_df[4][1],4)
        if win_trade_num != 0:
            random_start_index = int(random()*(len(symbol_history_df)-100))
            side = 'buy' if randint(0,1) == 0 else 'sell'
            if side == 'buy':
                random_end_index = symbol_history_df[random_start_index:random_start_index+1000][4].idxmax()
            else:
                random_end_index = symbol_history_df[random_start_index:random_start_index+1000][4].idxmin()
            close_order_data[str(order_id_count)] = {
                'symbol': symbol,
                'side': side,
                'entry_price': symbol_history_df[4][random_start_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][random_start_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
                'exit_price': symbol_history_df[4][random_end_index],
                'exit_time': datetime.fromtimestamp(symbol_history_df[0][random_end_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'exit_quantity': quantity,
                'Realized_Pnl': round((symbol_history_df[4][random_end_index] - symbol_history_df[4][random_start_index])*quantity,2) if side == 'buy' else round((symbol_history_df[4][random_start_index] - symbol_history_df[4][random_end_index])*quantity,2),
                'Unrealized_Pnl': 0,
            }
            order_id_count += 1
            win_trade_num -= 1
        else:
            random_start_index = int(random()*(len(symbol_history_df)-100))
            side = 'buy' if randint(0,1) == 0 else 'sell'
            if side == 'buy':
                random_end_index = symbol_history_df[random_start_index:random_start_index+200][4].idxmin()
            else:
                random_end_index = symbol_history_df[random_start_index:random_start_index+200][4].idxmax()
            close_order_data[str(order_id_count)] = {
                'symbol': symbol,
                'side': side,
                'entry_price': symbol_history_df[4][random_start_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][random_start_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
                'exit_price': symbol_history_df[4][random_end_index],
                'exit_time': datetime.fromtimestamp(symbol_history_df[0][random_end_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'exit_quantity': quantity,
                'Realized_Pnl': round((symbol_history_df[4][random_end_index] - symbol_history_df[4][random_start_index])*quantity,2) if side == 'buy' else round((symbol_history_df[4][random_start_index] - symbol_history_df[4][random_end_index])*quantity,2),
                'Unrealized_Pnl': 0,
                
            }
            order_id_count += 1
    return close_order_data

In [3]:
binance_usdm = binanceusdm({
    'option': {
        "defaultType": "swap",
    },
})
entry_data = generate_open_order(binance_usdm)
order_data = Caculate_Unrealized_Pnl(entry_data,binance_usdm)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
1000XEC/USDT:USDT,1,sell,0.03329,2023-03-18 08:45:00,37160.9067,14,market,swap,binanceusdm,78.41
AGIX/USDT:USDT,2,buy,0.3049,2023-03-12 10:15:00,3214.4005,9,market,swap,binanceusdm,584.7
APE/USDT:USDT,3,buy,3.718,2023-03-12 06:30:00,266.809,11,market,swap,binanceusdm,94.45
APT/USDT:USDT,5,sell,14.394,2023-03-14 22:30:00,96.2371,11,market,swap,binanceusdm,229.53
AUDIO/USDT:USDT,6,buy,0.2137,2023-03-12 22:45:00,4559.9635,3,market,swap,binanceusdm,331.51
BAKE/USDT:USDT,7,sell,0.206,2023-03-20 17:15:00,6246.0962,1,market,swap,binanceusdm,99.31
BCH/USDT:USDT,8,buy,112.23,2023-03-12 15:45:00,8.8629,18,market,swap,binanceusdm,175.49
BLZ/USDT:USDT,10,buy,0.06622,2023-03-13 00:30:00,14708.0453,17,market,swap,binanceusdm,274.6
BTCDOM/USDT:USDT,11,sell,1684.0,2023-03-21 07:15:00,0.6966,16,market,swap,binanceusdm,33.72
CHR/USDT:USDT,12,buy,0.1328,2023-03-12 23:30:00,7451.5648,18,market,swap,binanceusdm,175.11


In [4]:
close_data = generate_close_order(binance_usdm,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,ALGO/USDT:USDT,buy,0.211,2023-01-13 06:00:00,5824.1118,15,market,swap,binanceusdm,0.2954,2023-02-08 19:00:00,5824.1118,491.56,0
2,1000XEC/USDT:USDT,sell,0.03186,2023-03-04 19:00:00,45808.5204,11,market,swap,binanceusdm,0.0258,2023-03-10 18:00:00,45808.5204,277.6,0
3,TRX/USDT:USDT,sell,0.06785,2023-03-13 14:00:00,18271.5147,10,market,swap,binanceusdm,0.06417,2023-03-16 00:00:00,18271.5147,67.24,0
4,BTC/USDT:USDT,buy,23730.0,2023-02-02 07:00:00,0.0604,21,market,swap,binanceusdm,25911.3,2023-03-14 23:00:00,0.0604,131.75,0
5,1000LUNC/USDT:USDT,buy,0.1659,2023-01-09 18:00:00,6863.418,7,market,swap,binanceusdm,0.2058,2023-02-03 16:00:00,6863.418,273.85,0
6,ALGO/USDT:USDT,sell,0.2094,2023-03-14 08:00:00,5824.1118,11,market,swap,binanceusdm,0.1991,2023-03-16 00:00:00,5824.1118,59.99,0
7,ALGO/USDT:USDT,sell,0.2313,2023-01-21 03:00:00,5824.1118,11,market,swap,binanceusdm,0.224,2023-03-03 09:00:00,5824.1118,42.52,0
8,ARPA/USDT:USDT,sell,0.03083,2023-03-12 08:00:00,38182.5124,10,market,swap,binanceusdm,0.03078,2023-03-12 09:00:00,38182.5124,1.91,0
9,TRX/USDT:USDT,sell,0.0626,2023-01-17 03:00:00,18271.5147,5,market,swap,binanceusdm,0.05882,2023-01-19 07:00:00,18271.5147,69.07,0
10,ALGO/USDT:USDT,buy,0.2015,2023-03-09 07:00:00,5824.1118,20,market,swap,binanceusdm,0.1803,2023-03-11 15:00:00,5824.1118,-123.47,0


In [5]:
close_data

{'1': {'symbol': 'ALGO/USDT:USDT',
  'side': 'buy',
  'entry_price': 0.211,
  'entry_time': '2023-01-13 06:00:00',
  'entry_quantity': 5824.1118,
  'entry_leverage': 15,
  'entry_type': 'market',
  'entry_class_SF': 'swap',
  'exchange': 'binanceusdm',
  'exit_price': 0.2954,
  'exit_time': '2023-02-08 19:00:00',
  'exit_quantity': 5824.1118,
  'Realized_Pnl': 491.56,
  'Unrealized_Pnl': 0},
 '2': {'symbol': '1000XEC/USDT:USDT',
  'side': 'sell',
  'entry_price': 0.03186,
  'entry_time': '2023-03-04 19:00:00',
  'entry_quantity': 45808.5204,
  'entry_leverage': 11,
  'entry_type': 'market',
  'entry_class_SF': 'swap',
  'exchange': 'binanceusdm',
  'exit_price': 0.0258,
  'exit_time': '2023-03-10 18:00:00',
  'exit_quantity': 45808.5204,
  'Realized_Pnl': 277.6,
  'Unrealized_Pnl': 0},
 '3': {'symbol': 'TRX/USDT:USDT',
  'side': 'sell',
  'entry_price': 0.06785,
  'entry_time': '2023-03-13 14:00:00',
  'entry_quantity': 18271.5147,
  'entry_leverage': 10,
  'entry_type': 'market',
  'e

In [6]:
bybit_ex = bybit({
    "option":{
        "defaultType":"swap",
    }
})
bybit_ex.load_markets()
entry_data = generate_open_order(bybit_ex)
order_data = Caculate_Unrealized_Pnl(entry_data,bybit_ex)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
ADA/USDT:USDT,1,buy,0.3296,2023-03-21 15:00:00,2940.3117,10,market,swap,bybit,148.19
ALPHA/USDT:USDT,2,sell,0.1183,2023-03-20 16:45:00,8737.4399,22,market,swap,bybit,55.48
BAND/USDT:USDT,3,sell,1.824,2023-03-20 18:45:00,524.090645,5,market,swap,bybit,33.54
BEL/USDT:USDT,4,buy,0.6124,2023-03-21 16:30:00,1505.7973,11,market,swap,bybit,49.99
BICO/USDT:USDT,5,buy,0.3909,2023-03-21 16:15:00,147.797619,24,market,swap,bybit,3.19
COTI/USDT:USDT,6,buy,0.06951,2023-03-21 16:30:00,13406.6229,22,market,swap,bybit,67.7
DOGE/USDT:USDT,7,buy,0.07048,2023-03-21 15:00:00,13557.4837,14,market,swap,bybit,69.14
ENJ/USDT:USDT,8,buy,0.384,2023-03-21 16:15:00,2389.4863,6,market,swap,bybit,46.59
ETH/USDT:USDT,9,sell,1818.94,2023-03-21 21:45:00,0.5687,9,market,swap,bybit,13.52
FET/USDT:USDT,10,sell,0.41775,2023-03-20 15:45:00,2461.5385,12,market,swap,bybit,46.03


In [7]:
close_data = generate_close_order(bybit_ex,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,IOTA/USDT:USDT,buy,0.194,2023-03-13 01:00:00,5934.7181,21,market,swap,bybit,0.2315,2023-03-18 11:00:00,5934.7181,222.55,0
2,1INCH/USDT:USDT,buy,0.521,2023-03-05 00:00:00,2604.1667,2,market,swap,bybit,0.5665,2023-03-19 16:00:00,2604.1667,118.49,0
3,1000BTT/USDT:USDT,sell,0.000744,2023-01-30 15:00:00,1607717.0418,20,market,swap,bybit,0.000591,2023-03-11 19:00:00,1607717.0418,245.98,0
4,OMG/USDT:USDT,sell,1.222,2023-01-19 11:00:00,974.6589,4,market,swap,bybit,1.216,2023-01-19 20:00:00,974.6589,5.85,0
5,1000BTT/USDT:USDT,sell,0.000694,2023-02-11 00:00:00,1113745.623601,2,market,swap,bybit,0.000591,2023-03-11 19:00:00,1113745.623601,114.72,0
6,JST/USDT:USDT,buy,0.02103,2023-01-09 17:00:00,50403.2258,25,market,swap,bybit,0.03215,2023-02-07 11:00:00,50403.2258,560.48,0
7,BLUR/USDT:USDT,sell,0.5798,2023-03-16 22:00:00,1303.781,2,market,swap,bybit,0.5403,2023-03-21 00:00:00,1303.781,51.5,0
8,BICO/USDT:USDT,buy,0.2681,2023-01-08 02:00:00,1997.036754,23,market,swap,bybit,0.4383,2023-01-30 20:00:00,1997.036754,339.9,0
9,JST/USDT:USDT,buy,0.02944,2023-02-26 16:00:00,39188.499032,15,market,swap,bybit,0.03064,2023-02-27 08:00:00,39188.499032,47.03,0
10,DYDX/USDT:USDT,buy,2.465,2023-02-10 21:00:00,913.242,18,market,swap,bybit,2.355,2023-02-13 17:00:00,913.242,-100.46,0


In [8]:
okex5_ex = okex5({
    "option":{
        "defaultType":"swap",
    }
})
okex5_ex.load_markets()
entry_data = generate_open_order(okex5_ex)
order_data = Caculate_Unrealized_Pnl(entry_data,okex5_ex)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
AGLD/USDT:USDT,1,buy,0.4099,2023-03-21 15:00:00,271.816241,3,market,swap,okex5,6.55
BAND/USDT:USDT,2,buy,1.66,2023-03-21 15:00:00,149.210073,8,market,swap,okex5,15.07
COMP/USDT:USDT,3,buy,42.74,2023-03-21 17:15:00,21.9106,13,market,swap,okex5,46.01
DASH/USDT:USDT,4,sell,58.99,2023-03-22 05:15:00,17.7336,2,market,swap,okex5,20.39
ETC/USDT:USDT-230324,5,buy,19.195,2023-03-21 15:00:00,47.1054,8,market,swap,okex5,67.36
JST/USDT:USDT,6,sell,0.02807,2023-03-20 07:15:00,315.147204,3,market,swap,okex5,0.33
KISHU/USDT:USDT,7,sell,0.0,2023-03-20 03:15:00,113.937851,14,market,swap,okex5,0.0
LINK/USDT:USDT,8,buy,6.873,2023-03-21 16:00:00,141.7234,13,market,swap,okex5,97.36
LTC/USDT:USDT-230331,9,buy,76.93,2023-03-21 15:00:00,11.9646,22,market,swap,okex5,52.29
LUNA/USDT:USDT,10,buy,1.3639,2023-03-21 17:15:00,184.647626,15,market,swap,okex5,12.78


In [9]:
close_data = generate_close_order(okex5_ex,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,BNT/USDT:USDT,buy,0.4281,2023-02-11 07:00:00,301.403278,7,market,swap,okex5,0.6083,2023-03-21 21:00:00,301.403278,54.31,0
2,ONT/USDT:USDT,buy,0.2291,2023-02-07 23:00:00,463.333207,21,market,swap,okex5,0.263,2023-03-18 08:00:00,463.333207,15.71,0
3,CORE/USDT:USDT,buy,2.10265,2023-03-15 20:00:00,246.1944,18,market,swap,okex5,2.44,2023-03-21 18:00:00,246.1944,83.05,0
4,NEAR/USDT:USDT,buy,1.78,2023-03-10 23:00:00,166.415578,1,market,swap,okex5,2.2,2023-03-18 20:00:00,166.415578,69.89,0
5,BNT/USDT:USDT,buy,0.4673,2023-03-16 22:00:00,335.738013,21,market,swap,okex5,0.6083,2023-03-21 21:00:00,335.738013,47.34,0
6,SLP/USDT:USDT,sell,0.00338,2023-02-08 05:00:00,338.523816,13,market,swap,okex5,0.00238,2023-03-10 18:00:00,338.523816,0.34,0
7,GMT/USDT:USDT,buy,0.432,2023-02-11 07:00:00,296.646463,10,market,swap,okex5,0.4933,2023-03-19 11:00:00,296.646463,18.18,0
8,NEAR/USDT:USDT,sell,2.61,2023-02-09 09:00:00,115.384164,11,market,swap,okex5,1.69,2023-03-10 18:00:00,115.384164,106.15,0
9,NEAR/USDT:USDT,sell,1.813,2023-03-11 14:00:00,380.361727,13,market,swap,okex5,1.704,2023-03-11 23:00:00,380.361727,41.46,0
10,NEAR/USDT:USDT,sell,2.564,2023-02-08 14:00:00,425.7131,13,market,swap,okex5,2.757,2023-02-08 22:00:00,425.7131,-82.16,0


In [29]:
binance_s_ex = bitfinex2({
})
binance_s_ex.load_markets()
def fliter_symbols_list(symbols_list:list,type:str,exchange:Exchange):
    '''
    This function is used to filter symbols list by type(swap,spot)
    '''
    filtered_symbols_list = []
    for symbol in symbols_list:
        if exchange.markets[symbol]['type'] == type:
            filtered_symbols_list.append(symbol)
    return filtered_symbols_list

fliter_symbols_list(binance_s_ex.symbols,'swap',binance_s_ex)

['AAVE/USDT:USDT',
 'ADA/USDT:USDT',
 'ALG/USDT:USDT',
 'AMP/USDT:USDT',
 'APE/USDT:USDT',
 'APT/USDT:USDT',
 'ATO/USDT:USDT',
 'AVAX/BTC:BTC',
 'AVAX/USDT:USDT',
 'AXS/USDT:USDT',
 'BTC/EURT:EURT',
 'BTC/USDT:USDT',
 'BTCDOM/USDT:USDT',
 'COMP/USDT:USDT',
 'CRV/USDT:USDT',
 'DOGE/USDT:USDT',
 'DOT/BTC:BTC',
 'DOT/USDT:USDT',
 'EGLD/USDT:USDT',
 'EOS/USDT:USDT',
 'ETC/USDT:USDT',
 'ETH/BTC:BTC',
 'ETH/EURT:EURT',
 'ETH/USDT:USDT',
 'EUR/USDT:USDT',
 'FIL/USDT:USDT',
 'FTM/USDT:USDT',
 'GALA/USDT:USDT',
 'GBP/USDT:USDT',
 'ICP/USDT:USDT',
 'IOT/USDT:USDT',
 'JASMY/USDT:USDT',
 'JPY/USDT:USDT',
 'KNC/USDT:USDT',
 'LINK/USDT:USDT',
 'LTC/BTC:BTC',
 'LTC/USDT:USDT',
 'MATIC/USDT:USDT',
 'MKR/USDT:USDT',
 'NEAR/USDT:USDT',
 'NEO/USDT:USDT',
 'OMG/USDT:USDT',
 'SAND/USDT:USDT',
 'SHIB/USDT:USDT',
 'SOL/BTC:BTC',
 'SOL/USDT:USDT',
 'STG/USDT:USDT',
 'SUSHI/USDT:USDT',
 'TESTBTC/TESTUSDT:TESTUSDT',
 'TESTDOT/TESTUSDT:TESTUSDT',
 'TESTMATIC/TESTUSDT:TESTUSDT',
 'TESTSOL/TESTUSDT:TESTUSDT',
 'TR

In [6]:
from ccxt import binanceusdm , binance

ex = binance({
    "options": {
        "defaultType": "spot",
        "fetchMarkets": ["spot"]
    },
})

ex.load_markets()
print(list(ex.symbols))

['1INCH/BTC', '1INCH/BUSD', '1INCH/USDT', '1INCHDOWN/USDT', '1INCHUP/USDT', 'AAVE/BKRW', 'AAVE/BNB', 'AAVE/BRL', 'AAVE/BTC', 'AAVE/BUSD', 'AAVE/ETH', 'AAVE/USDT', 'AAVEDOWN/USDT', 'AAVEUP/USDT', 'ACA/BTC', 'ACA/BUSD', 'ACA/USDT', 'ACH/BTC', 'ACH/BUSD', 'ACH/TRY', 'ACH/USDT', 'ACM/BTC', 'ACM/BUSD', 'ACM/USDT', 'ADA/AUD', 'ADA/BIDR', 'ADA/BKRW', 'ADA/BNB', 'ADA/BRL', 'ADA/BTC', 'ADA/BUSD', 'ADA/ETH', 'ADA/EUR', 'ADA/GBP', 'ADA/PAX', 'ADA/RUB', 'ADA/TRY', 'ADA/TUSD', 'ADA/USDC', 'ADA/USDT', 'ADADOWN/USDT', 'ADAUP/USDT', 'ADX/BNB', 'ADX/BTC', 'ADX/BUSD', 'ADX/ETH', 'ADX/USDT', 'AE/BNB', 'AE/BTC', 'AE/ETH', 'AERGO/BTC', 'AERGO/BUSD', 'AGI/BNB', 'AGI/BTC', 'AGI/ETH', 'AGIX/BTC', 'AGIX/BUSD', 'AGIX/TRY', 'AGIX/USDT', 'AGLD/BNB', 'AGLD/BTC', 'AGLD/BUSD', 'AGLD/USDT', 'AION/BNB', 'AION/BTC', 'AION/BUSD', 'AION/ETH', 'AION/USDT', 'AKRO/BTC', 'AKRO/BUSD', 'AKRO/USDT', 'ALCX/BTC', 'ALCX/BUSD', 'ALCX/USDT', 'ALGO/BIDR', 'ALGO/BNB', 'ALGO/BTC', 'ALGO/BUSD', 'ALGO/ETH', 'ALGO/PAX', 'ALGO/RUB', 'ALGO/

In [66]:
from faker import Faker

def generate_Users_data(usernum:int) -> list:
    print('Generating Users Data...')
    fake = Faker()
    users_data = []
    for i in range(usernum):
        print(f'Generating User {i+1}...')
        name = fake.name()
        user_id = name.lower().replace(' ','_')
        card_type = ''
        while card_type not in ['visa','mastercard','discover','jcb']:
            card_type = fake.credit_card_provider().lower()
        users_data.append({
            'user_id': user_id,
            'user_name': name,
            'user_email': name.split(' ')[1].lower() + '@gmail.com',
            'user_password': fake.password(),
            'user_details' : {
                'credit_card' : {
                    'card_number': fake.credit_card_number(card_type=str(card_type).lower().replace(' ','')),
                    'card_type': card_type,
                    'card_holder_name': name,
                    'card_expiry_date': fake.credit_card_expire(start='now', end='+10y', date_format='%m/%y'),
                    'card_cvv': fake.credit_card_security_code(card_type=str(card_type).lower().replace(' ','')),
            }
        }
        })
    return users_data

users_data = generate_Users_data(10)
users_data

Generating Users Data...
Generating User 1...
Generating User 2...
Generating User 3...
Generating User 4...
Generating User 5...
Generating User 6...
Generating User 7...
Generating User 8...
Generating User 9...
Generating User 10...


[{'user_id': 'edwin_juarez',
  'user_name': 'Edwin Juarez',
  'user_email': 'juarez@gmail.com',
  'user_password': '(agE4PNUq#',
  'user_details': {'credit_card': {'card_number': '2710422970747203',
    'card_type': 'mastercard',
    'card_holder_name': 'Edwin Juarez',
    'card_expiry_date': '02/30',
    'card_cvv': '629'}}},
 {'user_id': 'corey_bennett',
  'user_name': 'Corey Bennett',
  'user_email': 'bennett@gmail.com',
  'user_password': 'V8s^7Lob7#',
  'user_details': {'credit_card': {'card_number': '6507985321399088',
    'card_type': 'discover',
    'card_holder_name': 'Corey Bennett',
    'card_expiry_date': '11/29',
    'card_cvv': '514'}}},
 {'user_id': 'donald_willis',
  'user_name': 'Donald Willis',
  'user_email': 'willis@gmail.com',
  'user_password': 'w0C!VX@y#t',
  'user_details': {'credit_card': {'card_number': '6011484686343443',
    'card_type': 'discover',
    'card_holder_name': 'Donald Willis',
    'card_expiry_date': '05/24',
    'card_cvv': '241'}}},
 {'user_id